In [0]:
from pyspark.sql.functions import (
    avg,
    col,
    date_format,
    dayofweek,
    format_number,
    stddev,
    to_date,
    row_number,
)
from pyspark.sql.window import Window

In [0]:
# 1. Variáveis de Caminho
# Caminho de destino no Volume do Unity Catalog (UC)
UC_VOLUME_PATH = '/Volumes/airports_database/default/airports_database/'
CSV_FILE_NAME = 'airports-database.csv'
FINAL_UC_PATH = UC_VOLUME_PATH + CSV_FILE_NAME

# 2. Criar o DataFrame (DF) a partir do Volume UC
df_aeroportos = (spark.read
  .format("csv")
  .option("header", "true")
  .option("inferSchema", "true")
  .load(FINAL_UC_PATH)
)

print("\nDataFrame criado com sucesso!")


DataFrame criado com sucesso!


In [0]:
# 3. Enriquecer o DataFrame com colunas de data formatada e nomes de dia/mês
df_aeroportos = df_aeroportos.withColumn("date", to_date(col("time_hour"), "yyyy-MM-dd HH:mm:ss"))
df_aeroportos = df_aeroportos.withColumn("day_of_week_name", date_format(col("date"), "EEEE"))
df_aeroportos = df_aeroportos.withColumn("month_of_year_name", date_format(col("date"), "MMM"))

In [0]:
display(df_aeroportos.limit(5))

id,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour,name,date,day_of_week_name,month_of_year_name
0,2013,1,1,517.0,515,2.0,830.0,819,11.0,UA,1545,N14228,EWR,IAH,227.0,1400,5,15,2013-01-01T05:00:00.000Z,United Air Lines Inc.,2013-01-01,Tuesday,Jan
1,2013,1,1,533.0,529,4.0,850.0,830,20.0,UA,1714,N24211,LGA,IAH,227.0,1416,5,29,2013-01-01T05:00:00.000Z,United Air Lines Inc.,2013-01-01,Tuesday,Jan
2,2013,1,1,542.0,540,2.0,923.0,850,33.0,AA,1141,N619AA,JFK,MIA,160.0,1089,5,40,2013-01-01T05:00:00.000Z,American Airlines Inc.,2013-01-01,Tuesday,Jan
3,2013,1,1,544.0,545,-1.0,1004.0,1022,-18.0,B6,725,N804JB,JFK,BQN,183.0,1576,5,45,2013-01-01T05:00:00.000Z,JetBlue Airways,2013-01-01,Tuesday,Jan
4,2013,1,1,554.0,600,-6.0,812.0,837,-25.0,DL,461,N668DN,LGA,ATL,116.0,762,6,0,2013-01-01T06:00:00.000Z,Delta Air Lines Inc.,2013-01-01,Tuesday,Jan


### 1. Qual é o número total de voos no conjunto de dados? 

In [0]:
# Conta o número total de voos no DataFrame e exibe o resultado em uma tabela
total_voos = df_aeroportos.count()

display(spark.createDataFrame([(total_voos,)], ["Quantidade Total de Voos"]))

Quantidade Total de Voos
336776


### 2. Quantos voos foram cancelados? 
(Considerando que voos cancelados têm dep_time e arr_time nulos)

In [0]:
# Conta o número de voos cancelados, considerando como cancelados aqueles em que os horários de partida (dep_time) e chegada (arr_time) são nulos.
voos_cancelados = df_aeroportos.filter(df_aeroportos.dep_time.isNull() & df_aeroportos.arr_time.isNull()).count()

# Exibe o total de voos cancelados em formato de tabela.
display(spark.createDataFrame([(voos_cancelados,)], ["Quantidade de Voos Cancelados"]))

Quantidade de Voos Cancelados
8255


### 3.  Quais são os 5 aeroportos com maior número de pousos? 

In [0]:
# Agrupa o DataFrame pelo aeroporto de destino ("dest"), conta o número de pousos em cada aeroporto,
# ordena em ordem decrescente de quantidade, seleciona os 5 aeroportos com mais pousos,
# renomeia as colunas para exibição e exibe o resultado em formato de tabela.
top5_pousos = (
    df_aeroportos.groupBy("dest")
    .count()
    .orderBy("count", ascending=False)
    .limit(5)
    .withColumnRenamed("dest", "Aeroporto de Destino")
    .withColumnRenamed("count", "Quantidade de Pousos")
)

display(top5_pousos)

Aeroporto de Destino,Quantidade de Pousos
ORD,17283
ATL,17215
LAX,16174
BOS,15508
MCO,14082


### 4.  Qual é a rota mais frequente?
(par origin-dest)

In [0]:
# Agrupa o DataFrame pelas colunas de origem ("origin") e destino ("dest"), conta o número de voos para cada rota,
# ordena em ordem decrescente de quantidade, seleciona a rota mais frequente (maior quantidade de voos),
# renomeia as colunas para exibição e exibe o resultado em formato de tabela.
rota_mais_frequente = (
    df_aeroportos.groupBy("origin", "dest")
    .count()
    .orderBy("count", ascending=False)
    .limit(1)
    .withColumnRenamed("origin", "Aeroporto de Origem")
    .withColumnRenamed("dest", "Aeroporto de Destino")
    .withColumnRenamed("count", "Quantidade de Voos")
)

display(rota_mais_frequente)

Aeroporto de Origem,Aeroporto de Destino,Quantidade de Voos
JFK,LAX,11262


### 5.  Quais são as 5 companhias aéreas com maior tempo médio de atraso na chegada? 
(Exiba também o tempo)

In [0]:
# Agrupa o DataFrame por companhia aérea ("carrier") e nome ("name"), calcula o atraso médio na chegada para cada grupo,
# ordena em ordem decrescente de atraso médio, seleciona as 5 companhias com maior atraso médio,
# formata o valor do atraso médio para duas casas decimais e exibe o resultado em formato de tabela.
top5_atraso_chegada = (
    df_aeroportos.groupBy("carrier", "name")
    .agg(avg("arr_delay").alias("Atraso Médio na Chegada"))
    .orderBy(col("Atraso Médio na Chegada").desc())
    .limit(5)
    .withColumn("Atraso Médio na Chegada", col("Atraso Médio na Chegada").cast("double"))
    .withColumn("Atraso Médio na Chegada (min)", format_number(col("Atraso Médio na Chegada"), 2)
).drop("Atraso Médio na Chegada"))

display(top5_atraso_chegada_fmt)

carrier,name,Atraso Médio na Chegada (min)
F9,Frontier Airlines Inc.,21.92
FL,AirTran Airways Corporation,20.12
EV,ExpressJet Airlines Inc.,15.80
YV,Mesa Airlines Inc.,15.56
OO,SkyWest Airlines Inc.,11.93


### 6.  Qual é o dia da semana com maior número de voos?

In [0]:
# Agrupa o DataFrame pelo nome do dia da semana ("day_of_week_name"), conta o número de voos em cada dia,
# ordena em ordem decrescente de quantidade, seleciona o dia com maior número de voos,
# renomeia as colunas para exibição e exibe o resultado em formato de tabela.
dia_mais_voos = (
    df_aeroportos.groupBy("day_of_week_name")
    .count()
    .orderBy(col("count").desc())
    .limit(1)
    .withColumnRenamed("day_of_week_name", "Dia da Semana")
    .withColumnRenamed("count", "Quantidade de Voos")
)

display(dia_mais_voos)

Dia da Semana,Quantidade de Voos
Monday,50690


### 7.  Qual é a rota que teve o maior tempo de voo médio?
(air_time)

In [0]:
# Agrupa o DataFrame pelas colunas de origem ("origin") e destino ("dest"), calcula o tempo médio de voo ("air_time") para cada rota,
# ordena em ordem decrescente de tempo médio, seleciona a rota com maior tempo médio de voo,
# renomeia as colunas para exibição e exibe o resultado em formato de tabela.
rota_maior_tempo_medio_voo = (
    df_aeroportos.groupBy("origin", "dest")
    .agg(avg("air_time").alias("Tempo Médio de Voo"))
    .orderBy(col("Tempo Médio de Voo").desc())
    .limit(1)
    .withColumnRenamed("origin", "Aeroporto de Origem")
    .withColumnRenamed("dest", "Aeroporto de Destino")
    .withColumnRenamed("Tempo Médio de Voo", "Tempo Médio de Voo (min)")
)

display(rota_maior_tempo_medio_voo)

Aeroporto de Origem,Aeroporto de Destino,Tempo Médio de Voo (min)
JFK,HNL,623.0877192982456


### 8.  Para cada aeroporto de origem, qual é o aeroporto de destino mais comum?

In [0]:
# Agrupa o DataFrame pelas colunas de origem ("origin") e destino ("dest") e conta o número de voos para cada rota.
rotas_contagem = df_aeroportos.groupBy("origin", "dest").count()

# Define uma janela particionada por aeroporto de origem, ordenando pela contagem de voos em ordem decrescente.
window_spec = Window.partitionBy("origin").orderBy(col("count").desc())

# Adiciona uma coluna de ranking para cada rota dentro de cada origem, baseado na contagem de voos.
rotas_ranked = rotas_contagem.withColumn("rank", row_number().over(window_spec))

# Filtra para manter apenas a rota mais comum (rank 1) de cada aeroporto de origem,
# renomeia as colunas para exibição e remove a coluna de ranking.
destino_mais_comum_por_origem = rotas_ranked.filter(col("rank") == 1) \
    .withColumnRenamed("origin", "Aeroporto de Origem") \
    .withColumnRenamed("dest", "Aeroporto de Destino Mais Comum") \
    .withColumnRenamed("count", "Quantidade de Voos") \
    .drop("rank")

display(destino_mais_comum_por_origem)

Aeroporto de Origem,Aeroporto de Destino Mais Comum,Quantidade de Voos
EWR,ORD,6100
JFK,LAX,11262
LGA,ATL,10263


### 9. Quais são as 3 rotas que tiveram a maior variação no tempo médio de voo?
(air_time)

In [0]:
# Agrupa o DataFrame pelas colunas de origem ("origin") e destino ("dest"), calcula o desvio padrão do tempo de voo ("air_time") para cada rota,
# ordena em ordem decrescente de desvio padrão, seleciona as 3 rotas com maior variação no tempo de voo,
# renomeia as colunas para exibição, formata o valor do desvio padrão para duas casas decimais e exibe o resultado em formato de tabela.
rotas_maior_variacao_tempo_voo = (
    df_aeroportos.groupBy("origin", "dest")
    .agg(stddev("air_time").alias("Desvio Padrão do Tempo de Voo"))
    .orderBy(col("Desvio Padrão do Tempo de Voo").desc())
    .limit(3)
    .withColumnRenamed("origin", "Aeroporto de Origem")
    .withColumnRenamed("dest", "Aeroporto de Destino")
    .withColumnRenamed("Desvio Padrão do Tempo de Voo", "Maior Variação no Tempo de Voo (min)")
    .withColumn("Maior Variação no Tempo de Voo (min)", format_number(col("Maior Variação no Tempo de Voo (min)"), 2))
)

display(rotas_maior_variacao_tempo_voo)

Aeroporto de Origem,Aeroporto de Destino,Maior Variação no Tempo de Voo (min)
LGA,MYR,25.32
EWR,HNL,21.27
JFK,HNL,20.69


### 10.  Quais são as 3 rotas mais comuns que tiveram atrasos na chegada superiores a 30 minutos?

In [0]:
# Filtra os voos com atraso na chegada superior a 30 minutos,
# agrupa pelas colunas de origem ("origin") e destino ("dest"),
# conta o número de voos atrasados para cada rota,
# ordena em ordem decrescente de quantidade,
# seleciona as 3 rotas com maior número de voos com atraso > 30 min,
# renomeia as colunas para exibição e exibe o resultado em formato de tabela.
rotas_atraso_maior_30 = (
    df_aeroportos.filter(col("arr_delay") > 30)
    .groupBy("origin", "dest")
    .count()
    .orderBy(col("count").desc())
    .limit(3)
    .withColumnRenamed("origin", "Aeroporto de Origem")
    .withColumnRenamed("dest", "Aeroporto de Destino")
    .withColumnRenamed("count", "Quantidade de Voos com Atraso > 30 min")
)

display(rotas_atraso_maior_30)

Aeroporto de Origem,Aeroporto de Destino,Quantidade de Voos com Atraso > 30 min
LGA,ATL,1563
JFK,LAX,1286
LGA,ORD,1188
